In [3]:
# Optional: Upload files from local system
from google.colab import files
uploaded = files.upload()

# 1. Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='darkgrid')
%matplotlib inline

Saving big_mart_sales.csv to big_mart_sales.csv


In [8]:
df = pd.read_csv('big_mart_sales.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [9]:
# Scenario 1: Complex Filtering with Multiple Conditions
# Sales > 90th percentile
sales_90th = df['Item_Outlet_Sales'].quantile(0.9)
# MRP within ±10% of its mean
mrp_mean = df['Item_MRP'].mean()
# Visibility below median
visibility_median = df['Item_Visibility'].median()

# Apply combined filter
filtered_df = df[
    (df['Item_Outlet_Sales'] > sales_90th) &
    (np.isclose(df['Item_MRP'], mrp_mean, rtol=0.1)) &
    (df['Item_Visibility'] < visibility_median)
]

filtered_df.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
54,NCP18,12.15,Low Fat,0.028760,Household,151.4708,OUT017,2007,NaN,Tier 2,Supermarket Type1,4815.0656
844,FDV01,NaN,Regular,0.000000,Canned,154.6314,OUT027,1985,Medium,Tier 3,Supermarket Type3,6515.5188
1173,FDH57,NaN,Low Fat,0.035574,Fruits and Vegetables,131.4284,OUT027,1985,Medium,Tier 3,Supermarket Type3,5404.9644
2355,FDR15,9.30,Regular,0.033432,Meat,153.1314,OUT035,2004,Small,Tier 2,Supermarket Type1,4653.9420
2863,FDS16,NaN,Regular,0.000000,Frozen Foods,145.2760,OUT027,1985,Medium,Tier 3,Supermarket Type3,5273.1360


In [10]:
# Scenario 2: Custom Aggregation Functions
summary_df = df.groupby('Outlet_Type')['Item_Outlet_Sales'].agg([
    ('Mean_Sales', 'mean'),
    ('StdDev_Sales', 'std'),
    ('25th_Percentile', lambda x: np.percentile(x, 25)),
    ('75th_Percentile', lambda x: np.percentile(x, 75)),
    ('High_Sales_Count', lambda x: (x > 5000).sum())
])

summary_df

,Mean_Sales,StdDev_Sales,25th_Percentile,75th_Percentile,High_Sales_Count
Outlet_Type,,,,,
Grocery Store,339.828500,260.851582,153.79980,458.73620,0
Supermarket Type1,2316.181148,1515.965558,1151.16820,3135.91800,359
Supermarket Type2,1995.498739,1375.932889,981.55565,2702.64865,39
Supermarket Type3,3694.038558,2127.760054,2044.33890,4975.52340,231


In [11]:
# Scenario 3: Row-wise Operations and Conditional Tagging
sales_90th = df['Item_Outlet_Sales'].quantile(0.9)
mrp_75th = df['Item_MRP'].quantile(0.75)

df['Performance_Tag'] = np.where(
    (df['Item_Outlet_Sales'] > sales_90th) & (df['Item_MRP'] > mrp_75th),
    'High Performer',
    'Average'
)

df[['Item_Outlet_Sales', 'Item_MRP', 'Performance_Tag']].head()

,Item_Outlet_Sales,Item_MRP,Performance_Tag
0,3735.1380,249.8092,Average
1,443.4228,48.2692,Average
2,2097.2700,141.6180,Average
3,732.3800,182.0950,Average
4,994.7052,53.8614,Average


In [12]:
# Scenario 4: Pivot Table & Reshaping
pivot_df = pd.pivot_table(
    df,
    values='Item_Outlet_Sales',
    index='Item_Type',
    columns='Outlet_Type',
    aggfunc='mean'
).unstack().fillna(0)

pivot_df.head()

Outlet_Type    Item_Type   
Grocery Store  Baking Goods    292.082544
               Breads          381.967442
               Breakfast       412.831042
               Canned          352.864879
               Dairy           341.866589
dtype: float64

In [13]:
# Scenario 5: Memory Optimization and Dtype Inference
memory_before = df.memory_usage(deep=True).sort_values(ascending=False)
top5_columns = memory_before.head(5).index.tolist()

for col in top5_columns:
    if df[col].dtype == 'float64':
        df[col] = df[col].astype('float32')
    elif df[col].dtype == 'int64':
        df[col] = df[col].astype('int32')
    elif df[col].dtype == 'object':
        df[col] = df[col].astype('category')

memory_after = df.memory_usage(deep=True).sort_values(ascending=False)

pd.DataFrame({
    'Before_MB': memory_before / (1024 ** 2),
    'After_MB': memory_after / (1024 ** 2)
})

,Before_MB,After_MB
Index,0.000126,0.000126
Item_Fat_Content,0.518250,0.008589
Item_Identifier,0.503946,0.503946
Item_MRP,0.065025,0.065025
Item_Outlet_Sales,0.065025,0.065025
Item_Type,0.555626,0.009682
Item_Visibility,0.065025,0.065025
Item_Weight,0.065025,0.065025
Outlet_Establishment_Year,0.065025,0.065025
Outlet_Identifier,0.512074,0.512074


In [14]:
#  Scenario 6: MultiIndex Operations
df_multi = df.set_index(['Outlet_Identifier', 'Item_Type'])
# All rows for OUT049
out049_data = df_multi.loc['OUT049']
# All rows for Dairy items
dairy_data = df_multi.xs('Dairy', level='Item_Type')
out049_data.head(), dairy_data.head()

(                      Item_Identifier  Item_Weight Item_Fat_Content  \
 Item_Type                                                             
 Dairy                           FDA15          9.3          Low Fat   
 Meat                            FDN15         17.5          Low Fat   
 Fruits and Vegetables           FDY07         11.8          Low Fat   
 Fruits and Vegetables           FDX32         15.1          Regular   
 Breakfast                       FDP49          9.0          Regular   
 
                        Item_Visibility  Item_MRP  Outlet_Establishment_Year  \
 Item_Type                                                                     
 Dairy                         0.016047  249.8092                       1999   
 Meat                          0.016760  141.6180                       1999   
 Fruits and Vegetables         0.000000   45.5402                       1999   
 Fruits and Vegetables         0.100014  145.4786                       1999   
 Breakfast    

In [ ]:
# Scenario 7: Exploding Lists and Normalizing Nested Structures
# Simulate a column with lists
df['Promo_Tags'] = [['summer', 'festival'], ['clearance'], np.nan, ['winter', 'sale'], ['festival']] * (len(df) // 5)
# Normalize by exploding the lists
df_exploded = df.explode('Promo_Tags')
df_exploded[['Item_Identifier', 'Promo_Tags']].head()
